<h2> An implementation of Associative Neural Network using Discrete HopField Net</h2>

In [ ]:
import numpy as np
import copy
import random

In [ ]:
input_list = [[1,1,-1,-1,-1,1],[1,-1,-1,1,-1,-1],[-1,-1,1,1,1,-1],[-1,1,1,-1,1,1]]

In [ ]:
#s = np.array(([1,1,-1,-1,-1,1],[1,-1,-1,1,-1,-1],[-1,-1,1,1,1,-1],[-1,1,1,-1,1,1]))
s = np.asarray(input_list)
t = s.T
print('Shapes- Input {}, Output {}'.format(s.shape,t.shape))

In [ ]:
dimensions = [s.shape[1],t.shape[0]]
print(dimensions)

In [ ]:
class NeuralNetwork:
    def __init__(self,dimensions):
        '''
        Initializing the weight vectors
        '''
        self.parameters = {}
        self.basin_state_dict = {}
        self.parameters['W'] = np.zeros([dimensions[0],dimensions[1]])
        print('Weight Matrix defined as {}'.format(self.parameters["W"].shape))
    
    def compute_weight(self,X,t):
        '''
        Weight is calculated using the Hebb's rule
        '''
        dw = np.dot(t,X)
        np.fill_diagonal(dw,0)
        dw = dw.astype(int)
        return dw
    
    def train(self,S,t):
        dw = self.compute_weight(S,t)
        self.parameters['W'] = self.parameters['W']+dw
    
    def compute_sum(self,order,y):
        W = self.parameters['W']
        total = 0;
        for i in range(0,len(y)):
            #print('Weight {}'.format(W[i][order]))
            total = total+(y[i]*W[i][order])
        
        return total;
                           
    def activate(self,yin,order,y):
        theta = 0;
        if yin > theta:
            yi = 1
        elif yin < theta:
            yi = -1
        elif yin == 0:
            y = y[order]
        
        y[order] = yi
        #print('post activate',y)
        return y
    
    def _check_basin_state(self,x,y,y_old,index):
        basin_map = self.basin_state_dict
        y_copy = copy.copy(y)
        if y != y_old:
            #print(x,y)
            if index in basin_map:
                basin_map[index].append(y_copy)
            else:
                basin_list=[]
                basin_list.append(y_copy)
                basin_map[index]=basin_list
        
        self.basin_state_dict = basin_map
        
    def compute_net_input(self,order,x,y,y_old,basin_index):
        #print('pre activate',y)
        total = self.compute_sum(order,y)
        yin = x[order]+total
        #print("order {},total {},x[order]: {},y[order]: {},yin: {},y-- {}".format(order,total,x[order],y[order],yin,y))
        y = self.activate(yin,order,y)
        #print("order {},total {},x[order]: {},y[order]: {},yin: {},y-- {}".format(order,total,x[order],y[order],yin,y))
        self._check_basin_state(x,y,y_old,basin_index)
        
        return y
                           
        
    def get_association(self,X,read_order):
        index = 0;
        assocations = []
        while index < len(X):
            x = X[index]
            #x = [-1,-1,1,1,1,1]
            y = copy.copy(x)
            #print("Before Index->{}.....x: {}, y: {}".format(index,x,y)) 
            repeat_counter = 0
            random_number_list = []
            while repeat_counter < 1:
                y_old = copy.copy(y)
                random_number_list.clear()
                for i in range(len(x)):
                    '''
                    rand_flag = True
                    while rand_flag == True:
                        rand_num =random.randint(0,5)
                        if rand_num not in random_number_list:
                            random_number_list.append(rand_num)
                            rand_flag = False
                        else:
                            #print('repeat',rand_num)
                            rand_flag = True
                    #print('index',rand_num,random_number_list)
                    '''
                    rand_num =random.randint(0,5)
                    y = self.compute_net_input(read_order[rand_num],x,y,y_old,index)
                if y != y_old:
                    #print('unstable.. y_old {},y_changed {}, x_input {},index {},repeat_counter {},random {}'.format(y_old,y,x,i,repeat_counter,rand_num))
                    print('unstable.. y_old {},y_changed {}, x_input {}'.format(y_old,y,x))
                    repeat_counter = 0
                else:
                    #print('stable.. y_old {},y_changed {}, x_input {},index {},repeat_counter {},random {}'.format(y_old,y,x,i,repeat_counter,rand_num))
                    print('stable.. y_old {},y_changed {}, x_input {}'.format(y_old,y,x))
                    repeat_counter = 1
                     
        
            #print("After  Index->{}.....x: {}, y: {}".format(index,x,y))                             
            assocations.append(y)
            index = index+1
        return assocations

In [ ]:
def combinationsByKFlip(vector,start,k,end,combination_list,result_list):
    '''
    Create bipolar input vector set
    '''
    if vector not in combination_list:
        combination_list.insert(start,vector)
        
    
    if k == 0:
        result_list.append(vector)
        return result_list
    
    for i in range(start,end):
        vector = _flipElement(vector,i)
        combinationsByKFlip(vector,i+1,k-1,end,combination_list,result_list)
        vector =  combination_list[start]
   
    return result_list


def _flipElement(vector,i):
    vectorCopy = copy.copy(vector)
    if vector[i] == 1:
        vectorCopy[i] = -1
    elif vector[i] == -1:
        vectorCopy[i] = 1
    elif vector[i] == 0:
        vectorCopy[i] = 0
    return vectorCopy

In [ ]:
input_vector = [1,1,-1,-1,-1,1]
bipolar_input = []
for i in range(0,len(input_vector)+1):
    start = 0
    end = len(input_vector)
    combination_list = []
    result_list = []
    result = combinationsByKFlip(input_vector,start,i,end,combination_list,result_list)
    bipolar_input.extend(result)


<b>Helper Functions to create list distinctions and validations</b>

In [ ]:
#Validate if all the training data are present

def validate_list(input_list,generated_list):
    #print(generated_list)
    for x in map(lambda u:u,filter(lambda x:x in input_list,generated_list)):
        print (x)

def check_stored_states(input_list,generated_list):
    stored_collection = []
    for x in map(lambda u:u,filter(lambda x:x in input_list,generated_list)):
        stored_collection.append(x)
    
    return stored_collection

def retreive_collections(input_list, association_list, splitby=None):
    '''
    function to split based on equiibrium or order or basin
    '''
    equilibrium = []
    spurious = []  
    if splitby == 'equilibrium':
        for x in association_list:
            if x in input_list:
                #print(x)
                equilibrium.append(x)
            else:
                spurious.append(x)
    else:
        pass
    
    return equilibrium,spurious


def create_reversed_state(states):
    reversed_list=[]
    for state in states:
        temp = copy.copy(state)
        i = 0
        while i < len(temp):
            if(temp[i] == -1):
                temp[i] = 1
            else:
                temp[i] = -1
            
            i = i+1
        reversed_list.append(temp)
    return reversed_list

def edit_distance(input_list,bipolar_input):
    max_dist = 6
    dist_map = {}
    #print(len(bipolar_input))
    for state in bipolar_input:
        temp = 6
        
        for j in range(0,len(input_list)):
            dist = 0;
            
            for i in range(0,len(state)):
                #print('i',i,state,input_list[j])
                if state[i] != input_list[j][i]:
                    dist = dist+1
                    #print(state,input_list[j],dist)
            
            if dist < temp:
                #print('temp',temp)
                temp = dist
            #print(temp)
           
            dist_map[ " ".join(str(x) for x in state)] = temp
        
        if temp < max_dist:
            max_dist = temp
        #print('max',max_dist)
                                         
    return max_dist,dist_map

In [ ]:
validate_list(input_list,bipolar_input)

In [ ]:
f"Total bipolar input combinations..{len(bipolar_input)}"

In [ ]:
hopNet = NeuralNetwork(dimensions)
hopNet.train(s,t)
print('Weight Matrix \n{}'.format(hopNet.parameters['W']))

In [ ]:
read_order = [4,5,0,1,2,3]
associations_training_samples = hopNet.get_association(input_list,read_order)

In [ ]:
validate_list(input_list,associations_training_samples)

In [ ]:
associations_total_samples = hopNet.get_association(bipolar_input,read_order)

In [ ]:
#test_list = [[-1, 1, 1, 1, 1, -1],[-1, -1, 1, 1, 1, -1]]
equilbrium,spurious = retreive_collections(input_list,associations_total_samples,splitby='equilibrium')
print('Total number of states in the equilbrium state {}'.format(len(equilbrium)))
print('Total number of states in the spurious state {}'.format(len(spurious)))

In [ ]:
#reversed_states = create_reversed_state([[1, -1, -1, -1, -1, 1], [1, 1, -1, -1, -1, 1]])
reversed_states = create_reversed_state(spurious)
stored = check_stored_states(input_list,reversed_states)
#reversed_states = check_reversed_state(spurious)

In [ ]:
basin_map = hopNet.basin_state_dict

In [ ]:
for index in basin_map:
    print(index)
    last = len(basin_map[index])-1
    print(basin_map[index][last])
    print(basin_map[index])

In [ ]:
len(basin_map)

In [ ]:
convergence_map = {}
for index in basin_map:
    convergence_index = 0;
    for state in basin_map[index]:
        if state not in input_list:
            convergence_index = convergence_index+1
        else:
            convergence_map[index] = (convergence_index,state,input_list[input_list.index(state)],input_list.index(state))
            break

In [ ]:
convergence_freq = {}
for key in convergence_map:
    val = convergence_map[key]
    #print(val[0],val[3],val[1])
    field = val[3]
    if field not in convergence_freq:
        count = 1
        convergence_freq[field] = count
    else:
        convergence_freq[field] = convergence_freq[field]+1
        #print(field,convergence_freq[field])

In [ ]:
convergence_freq

In [ ]:
max_dist,dist_map = edit_distance(input_list,bipolar_input)

In [ ]:
max_dist,dist_map